In [3]:
import numpy as np,pandas as pd, sys, os, re, zipfile, shutil

In [70]:
censusDF = pd.read_csv('./censusData.csv')
varDF = pd.read_csv("./acs2019Variables.csv", skiprows=[1,2,3])
pd.options.display.max_colwidth=1000
#change state and county fip codes to strings with standard lengths
censusDF['state'] = censusDF['state'].astype(str).str.zfill(2)
censusDF['county'] = censusDF['county'].astype(str).str.zfill(3)
#join these to create the combined fips code
censusDF['fips'] = censusDF['state']+censusDF['county']
#create a list of descriptive variable names in the same order as in censusDF
a = []
for i in censusDF.columns[:-4]:
    vName = varDF['label'][varDF['name']==i].to_string(index=False)+varDF['concept'][varDF['name']==i].to_string(index=False)
    a.append(vName)
for i in censusDF.columns[-4:]:
    a.append(i)
#rebuild dataframe with these column names
# censusDF = pd.DataFrame(censusDF.values, columns=a)
#et voila

In [51]:
path = r'../covid/processed_data/county_merged_parts/'
listFiles = []
for r, d, files in os.walk(path):
    for file in files:
        listFiles.append(file)

In [53]:
listFiles[0]

'Abbeville_SouthCarolina_UnitedStates.pkl'

In [65]:
testDF = pd.DataFrame()
for i,file in enumerate(listFiles):
    inDF = pd.read_pickle(f'{path}{file}')
    inDF.reset_index(level=0,inplace=True)
    testDF = pd.concat([testDF,inDF],ignore_index=True)
# test = f'{path}{listFiles[0]}'
# testDF= pd.read_pickle(test)

In [66]:
testDF

,dates,JHU_ConfirmedCases.data,JHU_ConfirmedCases.missing,NYT_ConfirmedCases.data,NYT_ConfirmedCases.missing,JHU_ConfirmedDeaths.data,JHU_ConfirmedDeaths.missing,JHU_ConfirmedRecoveries.data,JHU_ConfirmedRecoveries.missing,NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.data,...,AverageWindDirection.data,AverageWindDirection.missing,AverageWindSpeed.data,AverageWindSpeed.missing,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,fips,LND110210
0,2020-01-01,0.0,0,0.0,100,0.0,0,0.0,100,0.0,...,228.333333,0.0,7.750000,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
1,2020-01-02,0.0,0,0.0,100,0.0,0,0.0,100,0.0,...,149.583333,0.0,4.452083,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
2,2020-01-03,0.0,0,0.0,100,0.0,0,0.0,100,0.0,...,117.741402,0.0,3.056548,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
3,2020-01-04,0.0,0,0.0,100,0.0,0,0.0,100,0.0,...,235.208333,0.0,10.520833,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
4,2020-01-05,0.0,0,0.0,100,0.0,0,0.0,100,0.0,...,245.000000,0.0,8.250000,0.0,6.0,25.0,25.0,24527.0,45001.0,490.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122225,2021-10-11,349.0,0,0.0,100,9.0,0,0.0,100,0.0,...,149.583333,0.0,12.583333,0.0,1.0,8.0,8.0,2756.0,46137.0,1961.27
2122226,2021-10-12,349.0,0,0.0,100,9.0,0,0.0,100,0.0,...,53.750000,0.0,18.333333,0.0,1.0,8.0,8.0,2756.0,46137.0,1961.27
2122227,2021-10-13,349.0,0,0.0,100,9.0,0,0.0,100,0.0,...,202.916667,0.0,25.125000,0.0,1.0,8.0,8.0,2756.0,46137.0,1961.27
2122228,2021-10-14,349.0,0,0.0,100,9.0,0,0.0,100,0.0,...,282.083333,0.0,18.958333,0.0,1.0,8.0,8.0,2756.0,46137.0,1961.27


In [67]:
# mask = testDF['fips'].isnull()
testDF.dropna(subset=['fips'], inplace=True)

In [68]:
testDF['fips']=testDF['fips'].astype(int).astype(str)

In [69]:
testDF['fips'] = testDF['fips'].str.zfill(5)

In [71]:
newDF = testDF.join(censusDF.set_index('fips'),on='fips',how='left')

0          45001
1          45001
2          45001
3          45001
4          45001
           ...  
2122225    46137
2122226    46137
2122227    46137
2122228    46137
2122229    46137
Name: fips, Length: 1876326, dtype: object

In [29]:
a = newDF.isna().any()
colMissing = newDF.columns[a]
for m,n in enumerate(colMissing):
    print(m,n)

0 NYT_AllCausesDeathsMonthly_Deaths_AllCauses.data
1 NYT_AllCausesDeathsMonthly_Deaths_AllCauses.missing
2 NYT_AllCausesDeathsMonthly_Excess_Deaths.data
3 NYT_AllCausesDeathsMonthly_Excess_Deaths.missing
4 NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.data
5 NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.missing
6 TotalPopulation.data
7 TotalPopulation.missing
8 MaleAndFemale_AtLeast65_Population.data
9 MaleAndFemale_AtLeast65_Population.missing
10 Male_Total_Population.data
11 Male_Total_Population.missing
12 Female_Total_Population.data
13 Female_Total_Population.missing
14 MaleAndFemale_Under18_Population.data
15 MaleAndFemale_Under18_Population.missing
16 AverageDailyTemperature.data
17 AverageDailyTemperature.missing
18 AverageDewPoint.data
19 AverageDewPoint.missing
20 AverageRelativeHumidity.data
21 AverageRelativeHumidity.missing
22 AverageSurfaceAirPressure.data
23 AverageSurfaceAirPressure.missing
24 AveragePrecipitationTotal.data
25 AveragePrecipitationTotal.mis

In [74]:
newDF.to_csv('covidCensus.gz',index=False,compression='gzip')

In [36]:
anotherDF = pd.read_csv('covCens.gz')

In [25]:
path = r'../air_quality/pm25/'
listFiles = []
pm25DF = pd.DataFrame()
for r, d, files in os.walk(path):
    print(files)
    for file in files:
        indf = pd.read_csv(f'{path}{file}')
        pm25DF = pd.concat([pm25DF,indf],ignore_index=True)

['daily_pm25_speciation_2019_FIPS.csv', 'daily_pm25_speciation_2020_FIPS.csv', 'daily_pm25_speciation_2021_FIPS.csv']


In [32]:
pm25DF['fips'] = pm25DF['fips'].astype(str).str.zfill(5)

In [39]:
anotherDF['fips'] = anotherDF['fips'].astype(str)

In [45]:
anotherDF.columns

Index(['JHU_ConfirmedCases.data', 'JHU_ConfirmedCases.missing',
       'NYT_ConfirmedCases.data', 'NYT_ConfirmedCases.missing',
       'JHU_ConfirmedDeaths.data', 'JHU_ConfirmedDeaths.missing',
       'JHU_ConfirmedRecoveries.data', 'JHU_ConfirmedRecoveries.missing',
       'NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.data',
       'NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.missing',
       'NYT_ConfirmedDeaths.data', 'NYT_ConfirmedDeaths.missing',
       'NYT_AllCausesDeathsWeekly_Excess_Deaths.data',
       'NYT_AllCausesDeathsWeekly_Excess_Deaths.missing',
       'NYT_AllCausesDeathsWeekly_Deaths_AllCauses.data',
       'NYT_AllCausesDeathsWeekly_Deaths_AllCauses.missing',
       'NYT_AllCausesDeathsMonthly_Deaths_AllCauses.data',
       'NYT_AllCausesDeathsMonthly_Deaths_AllCauses.missing',
       'NYT_AllCausesDeathsMonthly_Excess_Deaths.data',
       'NYT_AllCausesDeathsMonthly_Excess_Deaths.missing',
       'NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.

In [40]:
newDF = anotherDF.merge(pm25DF.set_index('fips'),on='fips',how='left')

KeyboardInterrupt: 

In [3]:
testDF = pd.DataFrame()
for file in listFiles:
    inDF = pd.read_pickle(f'{path}{file}')
    testDF = pd.concat([testDF,inDF],ignore_index=True)

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['daily_pm25_speciation_2019_FIPS.csv', '__MACOSX/._daily_pm25_speciation_2019_FIPS.csv']

In [12]:
cool =  []
jack = os.walk(r'../air_quality/pm25')
for a,b,c in jack:
    for i in c:
        cool.append(i)
for i in cool:
    indf = pd.read_csv(f)

['daily_pm25_speciation_2019_FIPS.csv',
 'daily_pm25_speciation_2020_FIPS.csv',
 'daily_pm25_speciation_2021_FIPS.csv']

In [36]:
joke = os.walk(r'../air_quality/pm25/daily_pm25_speciation_2019_FIPS.csv')
for a,b,c in joke:
    print(c)

['daily_pm25_speciation_2019_FIPS.csv']
